In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sqlalchemy.dialects
from sqlalchemy.dialects import registry
from sqlalchemy.orm import sessionmaker
import seaborn as sns

In [7]:
# Define and import from CME ftp
file_name = "ftp://ftp.cmegroup.com/pub/settle/nymex_option.csv"

In [8]:
# Create Dataframe
df = pd.read_csv(file_name, index_col=False)
df_opts = pd.DataFrame (df,columns = ["PRODUCT SYMBOL", "CONTRACT MONTH", "CONTRACT YEAR", "PUT/CALL", "STRIKE", "SETTLE", "PT CHG", "EST. VOL", "PRIOR SETTLE", "PRIOR VOL", "PRIOR INT", "TRADEDATE"])

# Select Columns
df.drop (["CONTRACT DAY", "CONTRACT", "OPEN", "HIGH", "PRODUCT DESCRIPTION", "HIGH AB INDICATOR", 
            "LOW", "LOW AB INDICATOR", "LAST", "LAST AB INDICATOR"
            ], axis = 1, inplace = True)

# Remove display delimiters
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Convert NaNs to numeric format
df_opts["EST. VOL"] = pd.to_numeric(df_opts["EST. VOL"], errors="coerce")
df_opts["PT CHG"] = pd.to_numeric(df_opts["PT CHG"], errors="coerce")
df_opts["PRIOR VOL"] = pd.to_numeric(df_opts["PRIOR VOL"], errors="coerce")
df_opts["PRIOR INT"] = pd.to_numeric(df_opts["PRIOR INT"], errors="coerce")

# Drop Null numerics
df_opts = df_opts.dropna(subset=["EST. VOL"])
df_opts = df_opts.dropna(subset=["PRIOR VOL"])
df_opts = df_opts.dropna(subset=["PRIOR INT"])

# Convert float to int
df_opts["EST. VOL"] = df_opts["EST. VOL"].astype(int)
df_opts["PRIOR VOL"] = df_opts["PRIOR VOL"].astype(int)
df_opts["PRIOR INT"] = df_opts["PRIOR INT"].astype(int)

# Parse product symbol to LN (European Gas Options)
df1 = df_opts[df_opts["PRODUCT SYMBOL"].str.contains("LN")]

# Remove empty estimated volume rows
df1.loc[df1["EST. VOL"] > 0]

# Return estimated volume in descending max to min order
df1.sort_values (by = ["EST. VOL"], inplace = True, ascending = False)
df1.head()

,PRODUCT SYMBOL,CONTRACT MONTH,CONTRACT YEAR,PUT/CALL,STRIKE,SETTLE,PT CHG,EST. VOL,PRIOR SETTLE,PRIOR VOL,PRIOR INT,TRADEDATE
20796,LN,1,2021,C,3.50,0.0833,0.0147,3486,0.0686,510,15447,11/24/2020
20663,LN,12,2020,P,2.75,0.0000,-0.0595,1645,0.0595,174,13570,11/24/2020
20960,LN,1,2021,P,2.75,0.1509,-0.0265,1584,0.1774,624,7200,11/24/2020
20952,LN,1,2021,P,2.50,0.0544,-0.0102,1332,0.0646,934,21302,11/24/2020
20461,LN,12,2020,C,2.75,0.0250,0.0045,1267,0.0205,4146,10378,11/24/2020


In [9]:
#Total Options Volume
Total_Volume = df1 ["EST. VOL"].sum()
Total_Prior = df1 ["PRIOR VOL"].sum()
Total_OI = df1 ["PRIOR INT"].sum()
print ("Yesterday's total options volume was", Total_Volume,"."" Prior volume was", Total_Prior, "while open interest stands at", Total_OI,".")

Yesterday's total options volume was 47463 . Prior volume was 102837 while open interest stands at 1332989 .


In [10]:
# Total Call Volume